In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession\
    .builder\
    .appName("twitter_silver")\
    .getOrCreate()

23/01/18 10:01:33 WARN Utils: Your hostname, river-virtual-machine resolves to a loopback address: 127.0.1.1; using 192.168.133.128 instead (on interface ens33)
23/01/18 10:01:33 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/01/18 10:01:35 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
df_tweet = spark.read.json("/home/river/Documentos/airflowalura/datalake/Silver/twitter_datascience/tweet")

In [4]:
df_tweet.printSchema()

root
 |-- author_id: string (nullable = true)
 |-- conversation_id: string (nullable = true)
 |-- created_at: string (nullable = true)
 |-- id: string (nullable = true)
 |-- like_count: long (nullable = true)
 |-- quote_count: long (nullable = true)
 |-- reply_count: long (nullable = true)
 |-- retweet_count: long (nullable = true)
 |-- text: string (nullable = true)
 |-- process_date: date (nullable = true)



In [5]:
from pyspark.sql import functions as f

In [11]:
tweet_conversas = df_tweet.alias("tweet")\
                   .groupBy(f.to_date("created_at").alias("created_date"))\
                    .agg(
                        f.countDistinct("author_id").alias("n_tweets"),
                        f.sum("like_count").alias("n_like"),
                        f.sum("quote_count").alias("n_quote"),
                        f.sum("reply_count").alias("n_reply"),
                        f.sum("retweet_count").alias("n_retweet")
                    ).withColumn("weekday", f.date_format("created_date", "E"))

In [13]:
tweet_conversas.show()

+------------+--------+------+-------+-------+---------+-------+
|created_date|n_tweets|n_like|n_quote|n_reply|n_retweet|weekday|
+------------+--------+------+-------+-------+---------+-------+
|  2022-11-29|      62|   100|      0|      5|    15062|    Tue|
|  2022-11-23|      42|   152|      1|      6|     2554|    Wed|
|  2022-11-28|      65|   243|      1|     14|     5763|    Mon|
|  2022-11-24|      60|   204|      1|      4|     3634|    Thu|
|  2022-11-27|      46|    20|      1|      3|     5538|    Sun|
|  2022-11-26|      72|    40|      0|      1|     5788|    Sat|
|  2022-11-25|      47|     8|      0|      1|     4510|    Fri|
+------------+--------+------+-------+-------+---------+-------+

